In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
from Tools import TrateNan, plot_predict, WOE, frecuencia
import dask


from sklearn.preprocessing import Imputer

import unicodedata
import re
import string
import nltk

from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
import pickle

from wordcloud import WordCloud, ImageColorGenerator
from scipy.misc import imread
from nltk.corpus import stopwords as sw

    
def elimina_tildes(s):
    return ''.join((c for c in unicodedata.normalize('NFD', unicode(s)) if unicodedata.category(c) != 'Mn')) 
def replace_punctuation ( text ):
    return re.sub('[%s]' % re.escape(string.punctuation), '', text)

%matplotlib inline

In [28]:
%%time
df=pd.read_csv("../../../Documentos/Data/Final TSC/train.csv").sample(frac=.1,
                                                    random_state=0).reset_index(drop=True)
df.head()

CPU times: user 2.89 s, sys: 120 ms, total: 3.01 s
Wall time: 3 s


%%time
df=dd.read_csv("../../../Documentos/Data/Final TSC/train.csv")

In [29]:
print df.shape

(210171, 10)


In [30]:
trate_nan=TrateNan(df)
trate_nan.count_nan()

,Columna,Cardinalidad,Num_NaN
1,EMPLOYER_NAME,50221.0,3.0
2,SOC_NAME,1166.0,1277.0
3,JOB_TITLE,43580.0,4.0
4,FULL_TIME_POSITION,2.0,0.0
5,PREVAILING_WAGE,18849.0,6.0
6,YEAR,6.0,0.0
7,WORKSITE,6949.0,0.0
8,lon,2400.0,7529.0
9,lat,2402.0,7529.0
10,ELIGIBLE,2.0,0.0


In [31]:
frecuencia(df, columns=["ELIGIBLE", "EMPLOYER_NAME", "SOC_NAME", "JOB_TITLE"])

  ELIGIBLE  Conteo  Frecuencia
0        1  182837    0.869944
1        0   27334    0.130056 


                       EMPLOYER_NAME  Conteo  Frecuencia
0                    INFOSYS LIMITED    9296    0.044231
1  TATA CONSULTANCY SERVICES LIMITED    4533    0.021568
2                      WIPRO LIMITED    3372    0.016044
3            DELOITTE CONSULTING LLP    2649    0.012604
4                      ACCENTURE LLP    2401    0.011424 


                            SOC_NAME  Conteo  Frecuencia
0          Computer Systems Analysts   20520    0.097635
1               Computer Programmers   15654    0.074482
2  SOFTWARE DEVELOPERS, APPLICATIONS   15637    0.074401
3          COMPUTER SYSTEMS ANALYSTS   15132    0.071999
4  Software Developers, Applications   13328    0.063415 


             JOB_TITLE  Conteo  Frecuencia
0   PROGRAMMER ANALYST   17316    0.082390
1    SOFTWARE ENGINEER    8592    0.040881
2  COMPUTER PROGRAMMER    4990    0.023743
3      SYSTEMS ANALYST    4362    0.020755

In [32]:
df.dropna(subset=["FULL_TIME_POSITION","SOC_NAME","JOB_TITLE"
                  ,"EMPLOYER_NAME"], inplace=True)

In [33]:
imputer=Imputer(strategy="mean")
imputer.fit(df[["lon","lat","PREVAILING_WAGE"]])
df[["lon","lat","PREVAILING_WAGE"]]=imputer.transform(df[["lon","lat","PREVAILING_WAGE"]])

In [34]:
df.EMPLOYER_NAME=df.EMPLOYER_NAME.map(replace_punctuation)

In [35]:
df.reset_index(drop=True, inplace=True)

texto=df.EMPLOYER_NAME.to_string(index=False).lower()

texto=texto.split()

stopwords=sw.words('english')

texto=[pal for pal in texto if pal not in stopwords]

freq=nltk.FreqDist(texto)

texto=[x for x,y in freq.items() if y >1]
stw=[x for x,y in freq.items() if y in range(1,5)]

stw.extend(stopwords)

In [36]:
stw=pickle.load(open("./X_words/stw.pkl","rb"))
texto=pickle.load(open("./X_words/texto.pkl","rb"))
freq=pickle.load(open("./X_words/freq.pkl","rb"))

In [37]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

cvct= CountVectorizer(max_features=2000,
                     stop_words=stw,
                     #tokenizer=texto,
                     ngram_range=(1,3))
cvct.fit(df.EMPLOYER_NAME)



CPU times: user 2.68 s, sys: 108 ms, total: 2.79 s
Wall time: 2.61 s


In [38]:
dtm=cvct.transform(df.EMPLOYER_NAME)

X_words=pd.DataFrame(dtm.toarray(),columns=cvct.get_feature_names(), )

%%time
aux_=np.zeros((0, 5000),)
for j in range(1):
    
    dtm=cvct.transform(df.EMPLOYER_NAME[48877*(j):48877*(j+1)])

    X_words=pd.DataFrame(dtm.toarray(),columns=cvct.get_feature_names(), )
    
    


In [ ]:
pd.concat([X_words, df], axis=1)

In [28]:
woe=WOE(data=df, target="ELIGIBLE")
woe.iv(["lat"], bins=10)

[('lat', 0.0026345261733769967)]

In [ ]:
woe.fit([])